## 8.6 補足：正則化

In [ ]:
library(caret)
library(tidyverse)
my_url <- str_c("https://raw.githubusercontent.com/taroyabuki",
                "/fromzero/master/data/wine.csv")
my_data <- read_csv(my_url)

### 8.6.1 正則化の実践

In [ ]:
A <- 2
B <- 0.1

my_model <- train(
  form = LPRICE2 ~ .,
  data = my_data,
  method = "glmnet",
  standardize = TRUE,
  tuneGrid = data.frame(
    lambda = A,
    alpha = B))

In [ ]:
coef(my_model$finalModel, A)
#>                         1
#> (Intercept) -2.8015519302
#> WRAIN        .
#> DEGREES      0.0832910512
#> HRAIN       -0.0004147386
#> TIME_SV      0.0023104647

In [ ]:
my_test <- data.frame(
  WRAIN = 500, DEGREES = 17,
  HRAIN = 120, TIME_SV = 2)
my_model %>% predict(my_test)
#> [1] -1.430752

### 8.6.2 ペナルティの強さと係数の関係

In [ ]:
library(ggfortify)
library(glmnetUtils)

my_data2 <- my_data %>% scale %>%
  as.data.frame

B <- 0.1

glmnet(
  form = LPRICE2 ~ .,
  data = my_data2,
  alpha = B) %>%
  autoplot(xvar = "lambda") +
  xlab("log A ( = log lambda)") +
  theme(legend.position = c(0.15, 0.25))

### 8.6.3 パラメータの決定

In [ ]:
As <- seq(0, 0.1, length.out = 21)
Bs <- seq(0, 0.1, length.out =  6)

my_model <- train(
  form = LPRICE2 ~ ., data = my_data, method = "glmnet", standardize = TRUE,
  trControl = trainControl(method = "LOOCV"),
  tuneGrid = expand.grid(lambda = As, alpha  = Bs))

my_model$bestTune
#>   alpha lambda
#> 8     0  0.035

In [ ]:
tmp <- "B ( = alpha)"
ggplot(my_model) +
  theme(legend.position = c(0, 1), legend.justification = c(0, 1)) +
  xlab("A ( = lambda)") +
  guides(shape = guide_legend(tmp), color = guide_legend(tmp))

In [ ]:
my_model$results %>%
  filter(RMSE == min(RMSE))
#>   alpha lambda      RMSE ...
#> 1     0 0.0595 0.3117092 ...

### 8.6.4 補足：RとPythonで結果を同じにする方法